 # python Script for fetching the insights Schema 

In [1]:
pip install requests


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests
import json
from collections import defaultdict
from datetime import datetime
import time
import statistics

In [14]:
#  start_time = time.time()
#  end_time = time.time()
#  print(f"⏱️ Response Time: {end_time - start_time:.3f} seconds")

In [19]:
RATE_LIMIT_URL = "https://api.github.com/rate_limit"
def log_rate_limit():
    print("\n📊 Checking GitHub API rate limits...")
    res = requests.get(RATE_LIMIT_URL, headers=HEADERS)
    if res.status_code == 200:
        data = res.json()
        core = data['rate']
        print(f"🧮 API Calls Used: {core['limit'] - core['remaining']} / {core['limit']}")
        print(f"⏳ Remaining Calls: {core['remaining']}")
        print(f"🕒 Resets At (Unix Timestamp): {core['reset']}")
    else:
        print("⚠️ Failed to fetch rate limit info.")

#  GraphQL User Insights

In [20]:
import requests
import os
import time
import json

GITHUB_TOKEN ="token"
if not GITHUB_TOKEN:
    raise EnvironmentError("GITHUB_TOKEN not set.")

GRAPHQL_URL = "https://api.github.com/graphql"
HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Content-Type": "application/json"
}


def run_query(query, variables=None):
    start = time.time()
    response = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": query, "variables": variables or {}})
    end = time.time()
    print(f"⏱️ GraphQL Query Time: {end - start:.2f}s")

    if response.status_code == 200:
        log_rate_limit()
        return response.json()
    else:
        raise Exception(f"Query failed: {response.status_code}\n{response.text}")


GRAPHQL_USER_QUERY = """
query($username: String!) {
  user(login: $username) {
    name
    login
    email
    avatarUrl
    bio
    location
    company
    contributionsCollection {
      contributionCalendar {
        totalContributions
        weeks {
          contributionDays {
            date
            contributionCount
          }
        }
      }
    }
    repositories(first: 5, isFork: false, orderBy: {field: UPDATED_AT, direction: DESC}) {
      nodes {
        name
        url
        languages(first: 5, orderBy: {field: SIZE, direction: DESC}) {
          edges {
            size
            node {
              name
            }
          }
        }
        defaultBranchRef {
          target {
            ... on Commit {
              history(first: 5) {
                edges {
                  node {
                    committedDate
                    additions
                    deletions
                    message
                    url
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
"""


def transform_schema(raw):
    user = raw['data']['user']
    heatmap = []
    for week in user['contributionsCollection']['contributionCalendar']['weeks']:
        for day in week['contributionDays']:
            heatmap.append({
                "date": day['date'],
                "commits": day['contributionCount'],
                "added": 0,  # Not available directly
                "removed": 0
            })

    language_map = {}
    repos_summary = []
    for repo in user['repositories']['nodes']:
        lines_added, lines_removed, commits = 0, 0, 0

        for edge in repo.get("defaultBranchRef", {}).get("target", {}).get("history", {}).get("edges", []):
            node = edge['node']
            lines_added += node['additions']
            lines_removed += node['deletions']
            commits += 1

        # Language summary
        for lang in repo['languages']['edges']:
            lang_name = lang['node']['name']
            language_map[lang_name] = language_map.get(lang_name, 0) + lang['size']

        repos_summary.append({
            "repoName": repo['name'],
            "repoUrl": repo['url'],
            "heatmap": [],
            "avgDailyCode": lines_added // 5 if commits else 0,
            "totalLines": {
                "added": lines_added,
                "removed": lines_removed
            },
            "commitVolume": {
                "added": lines_added,
                "removed": lines_removed,
                "commits": commits
            },
            "repoSummary": [{
                "repoId": "",
                "repoName": repo['name'],
                "repoLink": repo['url'],
                "linesAdded": lines_added,
                "linesRemoved": lines_removed,
                "linesContributed": lines_added - lines_removed,
                "commits": commits
            }]
        })

    return {
        "profile": {
            "firstname": user['name'].split(" ")[0] if user['name'] else "",
            "lastname": " ".join(user['name'].split(" ")[1:]) if user['name'] else "",
            "email": user['email'],
            "photoUrl": user['avatarUrl'],
            "bio": user['bio'],
            "location": user['location'],
            "institute": user['company'],
            "expertises": [],
            "languages": language_map
        },
        "overallStats": {
            "avgDailyCode": 0,  # Not directly available
            "activeDays": 0,
            "totalLines": {
                "added": sum(l['totalLines']['added'] for l in repos_summary),
                "removed": sum(l['totalLines']['removed'] for l in repos_summary),
                "repos": len(repos_summary)
            },
            "commitVolume": {
                "added": sum(l['commitVolume']['added'] for l in repos_summary),
                "removed": sum(l['commitVolume']['removed'] for l in repos_summary),
                "commits": sum(l['commitVolume']['commits'] for l in repos_summary)
            },
            "badges": [],
            "heatmap": heatmap,
            "recentActivity": []  # Can be fetched via events API
        },
        "projectWiseStats": [],
        "milestoneWiseStats": [],
        "byRepo": repos_summary
    }


def main():
    username = "oneranks"  # Change to your GitHub alias/username
    raw = run_query(GRAPHQL_USER_QUERY, {"username": username})
    data = transform_schema(raw)

    with open("userInsightsSchema.json", "w") as f:
        json.dump(data, f, indent=2)
    print("✅ userInsightsSchema.json written successfully")
    


if __name__ == "__main__":
    main()


⏱️ GraphQL Query Time: 1.35s

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 8 / 5000
⏳ Remaining Calls: 4992
🕒 Resets At (Unix Timestamp): 1747917646
✅ userInsightsSchema.json written successfully


 # Hybrid Approach (graph ql + rest)

In [17]:

# --------------------------- CONFIG ---------------------------
GITHUB_TOKEN = token  # Replace this with your actual GitHub token
REPO_OWNER = "oneranks"
REPO_NAME = "oneranks-ui-lib"

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

GRAPHQL_URL = "https://api.github.com/graphql"
REST_BASE = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}"


# --------------------------- HELPERS ---------------------------
def run_graphql_query(query, variables=None):
    start_time = time.time()
    res = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": query, "variables": variables or {}})
    end_time = time.time()
    res.raise_for_status()
    print(f"⏱️ Response Time of qraphql query: {end_time - start_time:.3f} seconds")
    log_rate_limit()
    return res.json()


def get_rest(endpoint):
    start_time = time.time()
    res = requests.get(endpoint, headers=HEADERS)
    end_time = time.time()
    res.raise_for_status()
    print(f"⏱️ Response Time of REST API request {endpoint}: {end_time - start_time:.3f} seconds")
    log_rate_limit()
    return res.json()


# --------------------------- FETCH DATA ---------------------------
def fetch_commit_history():
    query = f"""
    query {{
      repository(owner: \"{REPO_OWNER}\", name: \"{REPO_NAME}\") {{
        defaultBranchRef {{
          target {{
            ... on Commit {{
              history(first: 100) {{
                edges {{
                  node {{
                    committedDate
                    additions
                    deletions
                    author {{
                      name
                      email
                      user {{
                        login
                      }}
                    }}
                  }}
                }}
              }}
            }}
          }}
        }}
        url
        name
      }}
    }}
    """
    data = run_graphql_query(query)
    repo = data["data"]["repository"]
    commits = [edge["node"] for edge in repo["defaultBranchRef"]["target"]["history"]["edges"]]
    return commits, repo["url"], repo["name"]


def fetch_contributors():
    return get_rest(f"{REST_BASE}/stats/contributors")


def fetch_issues():
    return get_rest(f"{REST_BASE}/issues?state=all&per_page=100")


def fetch_milestones():
    return get_rest(f"{REST_BASE}/milestones?state=all")


def fetch_projects():
    try:
        projects = get_rest(f"{REST_BASE}/projects")
        project_data = []
        for project in projects:
            columns = get_rest(project["columns_url"])
            column_data = [{"name": col["name"], "card_count": len(get_rest(col["cards_url"]))} for col in columns]
            project_data.append({"name": project["name"], "columns": column_data})
        return project_data
    except Exception:
        return []  # Skip if not enabled or fails


# --------------------------- TRANSFORM ---------------------------
def transform_repo_insights(repo_name, repo_url, commit_history, contributors_stats, issue_data, milestone_data, project_data):
    heatmap_by_date = defaultdict(lambda: {"commits": 0, "added": 0, "removed": 0})
    total_add, total_del, total_commits = 0, 0, 0
    contributors = {}

    for commit in commit_history:
        date = commit['committedDate'][:10]
        added = commit['additions']
        removed = commit['deletions']
        heatmap_by_date[date]['commits'] += 1
        heatmap_by_date[date]['added'] += added
        heatmap_by_date[date]['removed'] += removed
        total_add += added
        total_del += removed
        total_commits += 1

    heatmap = [{"date": d, **stats} for d, stats in heatmap_by_date.items()]

    for contributor in contributors_stats:
        login = contributor['author']['login'] if contributor['author'] else "anonymous"
        total_lines_added = sum(week['a'] for week in contributor['weeks'])
        total_lines_removed = sum(week['d'] for week in contributor['weeks'])
        total_commits_user = sum(week['c'] for week in contributor['weeks'])

        contributors[login] = {
            "commits": total_commits_user,
            "linesAdded": total_lines_added,
            "linesRemoved": total_lines_removed,
            "avgRevisionsPerPR": 0,
            "issueStats": {}
        }

    overall = {
        "totalCommits": total_commits,
        "linesAdded": total_add,
        "linesRemoved": total_del,
        "activeContributors": len(contributors)
    }

    tag_counts = defaultdict(lambda: {"resolved": 0, "pending": 0})
    for issue in issue_data:
        state = "resolved" if issue["state"] == "closed" else "pending"
        for label in issue.get("labels", []):
            tag_counts[label["name"]][state] += 1

    by_tag = {
        tag: {
            "resolvedIssueCount": counts["resolved"],
            "pendingIssueCount": counts["pending"]
        }
        for tag, counts in tag_counts.items()
    }

    milestone_insights = {}
    for ms in milestone_data:
        total = ms["open_issues"] + ms["closed_issues"]
        if total == 0:
            continue
        milestone_insights[ms["title"]] = {
            "totalIssues": total,
            "contributors": {
                ms["creator"]["login"]: {
                    "assigned": total,
                    "resolved": ms["closed_issues"],
                    "contributionPct": round((ms["closed_issues"] / total) * 100, 2)
                }
            }
        }

    project_distribution = {}
    for project in project_data:
        dist = {"inQueue": 0, "inProgress": 0, "inReview": 0, "done": 0, "backlog": 0}
        for col in project.get("columns", []):
            name = col["name"].lower()
            if "queue" in name:
                dist["inQueue"] += col["card_count"]
            elif "progress" in name:
                dist["inProgress"] += col["card_count"]
            elif "review" in name:
                dist["inReview"] += col["card_count"]
            elif "done" in name:
                dist["done"] += col["card_count"]
            elif "backlog" in name:
                dist["backlog"] += col["card_count"]
        project_distribution[project["name"]] = dist
    insights = {
        "repoName": repo_name,
        "repoUrl": repo_url,
        "heatmap": heatmap,
        "avgDailyCode": round(total_add / max(len(heatmap), 1), 2),
        "totalLines": {
            "added": total_add,
            "removed": total_del
        },
        "contributors": contributors,
        "overall": overall,
        "crs": {
            "submitted": {},
            "assigned": {}
        },
        "issues": {
            "resolved": sum(1 for i in issue_data if i["state"] == "closed"),
            "pending": sum(1 for i in issue_data if i["state"] == "open"),
            "byTag": by_tag
        },
        "projectDistribution": project_distribution,
        "milestoneInsights": milestone_insights
    }
    

    return {
        "repoName": repo_name,
        "repoUrl": repo_url,
        "heatmap": heatmap,
        "avgDailyCode": round(total_add / max(len(heatmap), 1), 2),
        "totalLines": {
            "added": total_add,
            "removed": total_del
        },
        "contributors": contributors,
        "overall": overall,
        "crs": {
            "submitted": {},
            "assigned": {}
        },
        "issues": {
            "resolved": sum(1 for i in issue_data if i["state"] == "closed"),
            "pending": sum(1 for i in issue_data if i["state"] == "open"),
            "byTag": by_tag
        },
        "projectDistribution": project_distribution,
        "milestoneInsights": milestone_insights
    }


# --------------------------- MAIN ---------------------------
if __name__ == "__main__":
    main_start_time = time.time()
    commits, repo_url, repo_name = fetch_commit_history()
    contributors = fetch_contributors()
    issues = fetch_issues()
    milestones = fetch_milestones()
    projects = fetch_projects()
    
    print("Transformation of insights")
    
    start_time = time.time()
    insights = transform_repo_insights(
        repo_name=repo_name,
        repo_url=repo_url,
        commit_history=commits,
        contributors_stats=contributors,
        issue_data=issues,
        milestone_data=milestones,
        project_data=projects
    )
    end_time = time.time()
    print(f"⏱️ Transformation Time: {end_time - start_time:.3f} seconds")
    
    

    with open("repoInsights.json", "w") as f:
        json.dump(insights, f, indent=2)

    print("✅ Final /repoInsights schema saved to repoInsights.json")
    main_end_time = time.time()
    print(f"⏱️ Total execution Time: {main_end_time - main_start_time:.3f} seconds")


⏱️ Response Time of qraphql query: 1.648 seconds

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 4 / 5000
⏳ Remaining Calls: 4996
🕒 Resets At (Unix Timestamp): 1747917646
⏱️ Response Time of REST API request https://api.github.com/repos/oneranks/oneranks-ui-lib/stats/contributors: 0.960 seconds

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 5 / 5000
⏳ Remaining Calls: 4995
🕒 Resets At (Unix Timestamp): 1747917646
⏱️ Response Time of REST API request https://api.github.com/repos/oneranks/oneranks-ui-lib/issues?state=all&per_page=100: 1.137 seconds

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 6 / 5000
⏳ Remaining Calls: 4994
🕒 Resets At (Unix Timestamp): 1747917646
⏱️ Response Time of REST API request https://api.github.com/repos/oneranks/oneranks-ui-lib/milestones?state=all: 0.586 seconds

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 7 / 5000
⏳ Remaining Calls: 4993
🕒 Resets At (Unix Timestamp): 1747917646
Transformation of insights
⏱️ Transformatio

# Graph ql for repo insights

In [32]:
def run_graphql_query(query, variables=None):
    start_time = time.time()
    res = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": query, "variables": variables or {}})
    end_time = time.time()
    res.raise_for_status()
    print(f"⏱️ Response Time of qraphql query: {end_time - start_time:.3f} seconds")
    log_rate_limit()
    return res.json()


def transform_repo_insights(data):
    repo = data['data']['repository']
    
    repo_name = repo['name']
    repo_url = repo['url']
    
    # Heatmap and contributor stats
    commits = repo['defaultBranchRef']['target']['history']['edges']
    heatmap = defaultdict(lambda: {'date': '', 'commits': 0, 'added': 0, 'removed': 0})
    contributors = defaultdict(lambda: {'commits': 0, 'linesAdded': 0, 'linesRemoved': 0, 'prRevisions': [], 'issueStats': {}})
    total_added, total_removed = 0, 0

    for commit in commits:
        node = commit['node']
        date = node['committedDate'][:10]
        additions = node['additions']
        deletions = node['deletions']
        email = node['author']['email'] if node['author'] else 'unknown'
        
        heatmap[date]['date'] = date
        heatmap[date]['commits'] += 1
        heatmap[date]['added'] += additions
        heatmap[date]['removed'] += deletions
        
        contributors[email]['commits'] += 1
        contributors[email]['linesAdded'] += additions
        contributors[email]['linesRemoved'] += deletions
        
        total_added += additions
        total_removed += deletions

    total_commits = len(commits)
    avg_daily_code = round(total_added / len(heatmap), 2) if heatmap else 0

    # Pull Request Stats
    prs = repo['pullRequests']['nodes']
    for pr in prs:
        login = pr['author']['login'] if pr['author'] else 'unknown'
        email = f"{login}@example.com"
        reviews = pr['reviews']['totalCount']
        contributors[email]['prRevisions'].append(reviews)

    for c in contributors.values():
        if c['prRevisions']:
            c['avgRevisionsPerPR'] = round(statistics.mean(c['prRevisions']), 2)
        else:
            c['avgRevisionsPerPR'] = 0
        del c['prRevisions']

    # Issue Stats
    issues = repo['issues']['nodes']
    issue_tags = defaultdict(int)
    issue_state_counts = {'resolved': 0, 'pending': 0}
    for issue in issues:
        login = issue['author']['login'] if issue['author'] else 'unknown'
        email = f"{login}@example.com"
        state = issue['state']
        labels = issue['labels']['nodes']
        
        if state == "CLOSED":
            issue_state_counts['resolved'] += 1
        else:
            issue_state_counts['pending'] += 1
        
        for label in labels:
            issue_tags[label['name']] += 1

    return {
        "repoName": repo_name,
        "repoUrl": repo_url,
        "heatmap": sorted(heatmap.values(), key=lambda x: x['date']),
        "avgDailyCode": avg_daily_code,
        "totalLines": {
            "added": total_added,
            "removed": total_removed
        },
        "contributors": contributors,
        "overall": {
            "totalCommits": total_commits,
            "linesAdded": total_added,
            "linesRemoved": total_removed,
            "activeContributors": len(contributors)
        },
        "crs": {
            "submitted": {},  # Placeholder
            "assigned": {}    # Placeholder
        },
        "issues": {
            "resolved": issue_state_counts['resolved'],
            "pending": issue_state_counts['pending'],
            "byTag": dict(issue_tags)
        },
        "projectDistribution": {},   # Placeholder
        "milestoneInsights": {}      # Placeholder
    }

# Example usage:
# insights = transform_repo_insights(graphql_response)

GRAPHQL_USER_QUERY="""
query {
  repository(owner: "oneranks", name: "oneranks-ui-lib") {
    name
    url
    defaultBranchRef {
      target {
        ... on Commit {
          history(first: 100) {
            edges {
              node {
                committedDate
                additions
                deletions
                author {
                  name
                  email
                  user {
                    login
                  }
                }
              }
            }
          }
        }
      }
    }
    languages(first: 5, orderBy: {field: SIZE, direction: DESC}) {
      edges {
        size
        node {
          name
        }
      }
    }
    pullRequests(states: [OPEN, MERGED, CLOSED], first: 10) {
      totalCount
      nodes {
        number
        title
        state
        additions
        deletions
        changedFiles
        author {
          login
        }
        comments {
          totalCount
        }
        reviews {
          totalCount
        }
      }
    }
    issues(states: [OPEN, CLOSED], first: 10) {
      totalCount
      nodes {
        number
        title
        state
        labels(first: 5) {
          nodes {
            name
          }
        }
        author {
          login
        }
      }
    }
  }
}
"""


def main():
    username = "oneranks"  # Change to your GitHub alias/username
    raw = run_query(GRAPHQL_USER_QUERY)
    insights = transform_repo_insights(raw)
 

    with open("transformgraphqlrepoInsightsSchema.json", "w") as f:
        json.dump(insights, f, indent=2)
    print("✅ transformgraphqlrepSchema.json written successfully")


if __name__ == "__main__":
    main()


⏱️ GraphQL Query Time: 1.34s
✅ transformgraphqlrepSchema.json written successfully


In [23]:

def run_graphql_query(query, variables=None):
    start_time = time.time()
    res = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": query, "variables": variables or {}})
    end_time = time.time()
    res.raise_for_status()
    print(f"⏱️ Response Time of qraphql query: {end_time - start_time:.3f} seconds")
    log_rate_limit()
    return res.json()


def transform_repo_insights(data):
    repo = data['data']['repository']
    
    repo_name = repo['name']
    repo_url = repo['url']
    
    # Heatmap and contributor stats
    commits = repo['defaultBranchRef']['target']['history']['edges']
    heatmap = defaultdict(lambda: {'date': '', 'commits': 0, 'added': 0, 'removed': 0})
    contributors = defaultdict(lambda: {'commits': 0, 'linesAdded': 0, 'linesRemoved': 0, 'prRevisions': [], 'issueStats': {}})
    total_added, total_removed = 0, 0

    for commit in commits:
        node = commit['node']
        date = node['committedDate'][:10]
        additions = node['additions']
        deletions = node['deletions']
        email = node['author']['email'] if node['author'] else 'unknown'
        
        heatmap[date]['date'] = date
        heatmap[date]['commits'] += 1
        heatmap[date]['added'] += additions
        heatmap[date]['removed'] += deletions
        
        contributors[email]['commits'] += 1
        contributors[email]['linesAdded'] += additions
        contributors[email]['linesRemoved'] += deletions
        
        total_added += additions
        total_removed += deletions

    total_commits = len(commits)
    avg_daily_code = round(total_added / len(heatmap), 2) if heatmap else 0

    # Pull Request Stats
    prs = repo['pullRequests']['nodes']
    for pr in prs:
        login = pr['author']['login'] if pr['author'] else 'unknown'
        email = f"{login}@example.com"
        reviews = pr['reviews']['totalCount']
        contributors[email]['prRevisions'].append(reviews)

    for c in contributors.values():
        if c['prRevisions']:
            c['avgRevisionsPerPR'] = round(statistics.mean(c['prRevisions']), 2)
        else:
            c['avgRevisionsPerPR'] = 0
        del c['prRevisions']

    # Issue Stats
    issues = repo['issues']['nodes']
    issue_tags = defaultdict(int)
    issue_state_counts = {'resolved': 0, 'pending': 0}
    for issue in issues:
        login = issue['author']['login'] if issue['author'] else 'unknown'
        email = f"{login}@example.com"
        state = issue['state']
        labels = issue['labels']['nodes']
        
        if state == "CLOSED":
            issue_state_counts['resolved'] += 1
        else:
            issue_state_counts['pending'] += 1
        
        for label in labels:
            issue_tags[label['name']] += 1
            
    #cr stats
    
    submitted = defaultdict(list)
    assigned = defaultdict(list)

    for pr in repo['pullRequests']['nodes']:
        pr_id = pr['number']
        author = pr['author']['login'] if pr['author'] else 'unknown'
        submitted[author].append(pr_id)

    # Reviews
    for review in pr['reviews']['nodes']:
        reviewer = review['author']['login'] if review['author'] else 'unknown'
        assigned[reviewer].append(pr_id)

    # Requested reviewers
    for request in pr['reviewRequests']['nodes']:
        reviewer = request.get('requestedReviewer', {}).get('login')
        if reviewer:
            assigned[reviewer].append(pr_id)
    #milestone insights
    milestone_insights = {}
    for milestone in repo.get('milestones', {}).get('nodes', []):
        title = milestone['title']
        issues = milestone['issues']['nodes']
        total_issues = len(issues)
        contributor_stats = defaultdict(lambda: {'assigned': 0, 'resolved': 0})

        for issue in issues:
            login = issue['author']['login'] if issue['author'] else 'unknown'
            contributor_stats[login]['assigned'] += 1
            if issue['state'] == 'CLOSED':
                contributor_stats[login]['resolved'] += 1

    # Project Distribution
    project_distribution = defaultdict(lambda: defaultdict(int))

    projects_data = repo.get('projectsV2')
    print("project Distribution",projects_data)
    if projects_data and 'nodes' in projects_data:
        for project in projects_data['nodes'] or []:
            if not project:
                continue
            title = project.get('title', 'unknown-project')
            items = project.get('items', {}).get('nodes', [])
            for item in items or []:
                fields = item.get('fieldValues', {}).get('nodes', [])
                for field in fields or []:
                    stage = field.get('name')
                    if stage:
                        project_distribution[title][stage] += 1


    # Compute contribution percent
    for login, stats in contributor_stats.items():
        stats['contributionPct'] = round((stats['resolved'] / total_issues) * 100, 2) if total_issues else 0

    milestone_insights[title] = {
            'totalIssues': total_issues,
            'contributors': dict(contributor_stats)
        }


    return {
        "repoName": repo_name,
        "repoUrl": repo_url,
        "heatmap": sorted(heatmap.values(), key=lambda x: x['date']),
        "avgDailyCode": avg_daily_code,
        "totalLines": {
            "added": total_added,
            "removed": total_removed
        },
        "contributors": contributors,
        "overall": {
            "totalCommits": total_commits,
            "linesAdded": total_added,
            "linesRemoved": total_removed,
            "activeContributors": len(contributors)
        },
        "crs": {
            "submitted": submitted,  # Placeholder
            "assigned":assigned    # Placeholder
        },
        "issues": {
            "resolved": issue_state_counts['resolved'],
            "pending": issue_state_counts['pending'],
            "byTag": dict(issue_tags)
        },
        "projectDistribution": project_distribution,   # Placeholder
        "milestoneInsights": milestone_insights      # Placeholder
    }

# Example usage:
# insights = transform_repo_insights(graphql_response)

GRAPHQL_USER_QUERY="""query {
  repository(owner: "oneranks", name: "oneranks-ui-lib") {
    name
    url
    defaultBranchRef {
      target {
        ... on Commit {
          history(first: 100) {
            edges {
              node {
                committedDate
                additions
                deletions
                author {
                  name
                  email
                  user {
                    login
                  }
                }
              }
            }
          }
        }
      }
    }
    languages(first: 5, orderBy: {field: SIZE, direction: DESC}) {
      edges {
        size
        node {
          name
        }
      }
    }
    pullRequests(states: [OPEN, MERGED, CLOSED], first: 100) {
      totalCount
      nodes {
        number
        title
        state
        additions
        deletions
        changedFiles
        author {
          login
        }
        comments {
          totalCount
        }
        reviews(first: 100) {
          totalCount
          nodes {
            author {
              login
            }
            state
          }
        }
        reviewRequests(first: 100) {
          nodes {
            requestedReviewer {
              ... on User {
                login
              }
            }
          }
        }
      }
    }
    issues(states: [OPEN, CLOSED], first: 100) {
      totalCount
      nodes {
        number
        title
        state
        labels(first: 5) {
          nodes {
            name
          }
        }
        author {
          login
        }
      }
    }
    projectsV2(first: 10) {
      nodes {
        title
        items(first: 100) {
          nodes {
            fieldValues(first: 10) {
              nodes {
                ... on ProjectV2ItemFieldSingleSelectValue {
                  name
                }
              }
            }
          }
        }
      }
    }
    milestones(first: 10) {
      nodes {
        title
        issues(first: 100) {
          nodes {
            state
            author {
              login
            }
          }
        }
      }
    }
  }
}"""

def main():
    username = "oneranks"  # Change to your GitHub alias/username
    raw = run_query(GRAPHQL_USER_QUERY)
    insights = transform_repo_insights(raw)
 

    with open("onetransformgraphqlrepoInsightsSchema.json", "w") as f:
        json.dump(insights, f, indent=2)
    print("✅ 1transformgraphqlrepSchema.json written successfully")
    


if __name__ == "__main__":
    main()


⏱️ GraphQL Query Time: 2.59s

📊 Checking GitHub API rate limits...
🧮 API Calls Used: 8 / 5000
⏳ Remaining Calls: 4992
🕒 Resets At (Unix Timestamp): 1747917646
project Distribution {'nodes': [None]}
✅ 1transformgraphqlrepSchema.json written successfully
